# **Domain boundaries predictions**<br>
### Using ResDom to prediction the domain boundaries on the proteins where a hit was observed against the depolyerase domain database. The results will then be integrated with the PFAM alignments
## 1. Generating a hmm profiles fror the proteins in the depolymerase DB
## 2. Secondary structure predicted using SCRATCH
## 3. The actual prediction
## 4. Writting the fasta files
***

## 1. The Hhbuild command :


In [1]:
import os
import subprocess
from tqdm import tqdm
from random import sample
import time
import pandas as pd
from multiprocessing.pool import ThreadPool

path_trunc = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/truncated_msa"
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"

results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.csv", sep="\t", header = 0)
sequence_df = pd.read_csv(f"{path_db}/Results_III_sequences.csv", sep = "\t" ,names= ["index", "sequence"])

def path_msa(id_seq) :
    # PT 1 
    df_id = results_df[results_df["index_seq"]==int(id_seq)]
    query = df_id.head(1)["protein_name"].values[0]
    "From the protein seq_id, get a MSA"
    path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
    strain , prophage = query.split("__")[0], query.split("__")[1]
    path_tab = f"{path_decipher}/{strain}/mmseqs_out/{prophage}/{query}.MSA.a2m"
    return path_tab, id_seq

def truncate_msa(path_tab, id_seq):
    list_seq = ["".join(seq.split("\n")[1:]) for seq in open(path_tab).read().split(">")[1:]]
    list_seq_char = []
    for index_l, seq_list in enumerate(list_seq) :
        list_seq_char.append(list(seq_list))
    list_names = [seq.split("\n")[0] for seq in open(path_tab).read().split(">")[1:]]
    col_names = [str(n) for n in range(len(list_seq[0]))]
    df_msa = pd.DataFrame(list_seq_char,index = list_names, columns = col_names)
    query = df_msa.head(1)
    query.transpose()
    to_keep = []
    for index_ , col in query.items() :
        if col.values[0] != "-" :
            to_keep.append(index_)
    next_df = df_msa.filter(axis=1, items = to_keep)
    with open(f"{path_trunc}/{id_seq}.fasta","w") as outfile :
        for index, row in next_df.iterrows() :
            seq = []
            for l in row :
                seq.append(l)
            outfile.write(f">{index}\n{''.join(seq)}\n")

def generate_truncate(id_seq) :
    path_tab , id_seq = path_msa(id_seq)
    truncate_msa(path_tab, id_seq)
    
if __name__ == '__main__':
    with ThreadPool(10) as pool:
        pool.map(generate_truncate, sequence_df["index"],chunksize = int(len(sequence_df["index"])/10))

def build_hmm(id_seq) :
    path_msa = f"/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/truncated_msa/{id_seq}.fasta"
    path_hmm = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/hmm_profiles"
    out_path = f"{path_hmm}/{id_seq}.ohmm"
    # ***********************************
    build_cmd = f"hhmake -o {out_path} -i {path_msa} -cons -name {id_seq}"
    build_subprocess = subprocess.Popen (build_cmd , shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
    filter_out, filter_err = build_subprocess.communicate()
    print(filter_out , filter_err)
    
if __name__ == '__main__':
    with ThreadPool(10) as pool:
        pool.map(build_hmm, sequence_df["index"],chunksize = int(len(sequence_df["index"])/10))

            
# *******************************************************************************************
#!/bin/bash
#BATCH --job-name=Uniref_
#SBATCH --qos=long
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=60 
#SBATCH --mem=120gb 
#SBATCH --time=10-00:00:00 
#SBATCH --output=Uniref_%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3


python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/script_files/uniref.hmm.py

SyntaxError: invalid syntax (3137951511.py, line 54)

***
## 2. Scratch prediction

In [ ]:
import os
import random
import subprocess
from tqdm import tqdm
from random import sample
import time
import pandas as pd

path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_depolymerase= "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/20102022_session"
path_pdb = "/home/conchae/databases/pdb100_2020Mar11/pdb100_2020Mar11"
path_scratch = "/home/conchae/databases/Scratch/SCRATCH-1D_1.3/bin"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/scratch_prediction"

depolymerase_db = f"{path_depolymerase}/depolymerase_proteins.DB.fasta"

    # The command line for each prot from the Elites :
command_scratch = f"{path_scratch}/run_SCRATCH-1D_predictors.sh {depolymerase_db} {path_out} 60"
scratch_subprocess = subprocess.Popen (command_scratch , shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
scratch_out, scratch_err = scratch_subprocess.communicate()
print(scratch_out, scratch_err)


# **********************************************************************************************************
#!/bin/bash
#BATCH --job-name=scratch_
#SBATCH --qos=long
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=60 
#SBATCH --mem=150gb 
#SBATCH --time=12-00:00:00 
#SBATCH --output=scratch_%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate resdom

/home/conchae/databases/Scratch/SCRATCH-1D_1.3/bin/run_SCRATCH-1D_predictors.sh  /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/20102022_session/depolymerase_proteins.DB.fasta /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/scratch_prediction 60


> Organize the results into convenient files

In [ ]:
import os

path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/scratch_prediction"

scratch_acc = open(f"{path_out}/scratch_07022023.acc").read().split(">")[1:]
scratch_ss  = open(f"{path_out}/scratch_07022023.ss").read().split(">")[1:]

scratch_results = {".acc" : scratch_acc, ".ss" : scratch_ss}

for suffix in scratch_results :
    for index, seq in enumerate(scratch_results[suffix]) :
        idex_seq = seq.split("\n")[0]
        info_seq = seq.split("\n")[1]
        with open(f"{path_out}/{idex_seq}{suffix}","w") as outfile :
            outfile.write(f">{idex_seq}\n{info_seq}")


***
## 3. ESM prediction 


> Installing the version esm2

In [ ]:
python -m pip install transformers[esm2_t33_650M_UR50D] --ignore-installed certifi

>The prediction 

In [ ]:
python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/script_files/extract.py \
esm1_t34_670M_UR50S \
/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/Results_III_sequences.fasta \
/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/esm_prediction/esm.07022023.predict.out \
--truncation_seq_length 1022 \
--repr_layers 34 \
--include mean

***
# 4. The fasta files

In [ ]:
import os
import pandas as pd

path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/fasta_files"

sequence_df = pd.read_csv(f"{path_db}/Results_III_sequences.csv", sep = "\t" ,names= ["index", "sequence"])

for index, row in sequence_df.iterrows():
    with open(f"{path_out}/{row['index']}.fasta","w") as outfile :
        outfile.write(f">{row['index']}\n{row['sequence']}")
